In [13]:
import pandas as pd

df = pd.read_csv('final.csv')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18573 entries, 0 to 18572
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          18573 non-null  object 
 1   authors        18573 non-null  object 
 2   affiliations   18573 non-null  object 
 3   citedby        18573 non-null  float64
 4   mainterms      18573 non-null  object 
 5   subject_areas  18573 non-null  object 
 6   publisher      18562 non-null  object 
 7   Year           18573 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 1.1+ MB


In [11]:
df2 = pd.read_csv('finalscrap.csv')

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1000 non-null   object
 1   authors       1000 non-null   object
 2   affiliations  1000 non-null   object
 3   citedby       1000 non-null   int64 
 4   publisher     1000 non-null   object
 5   Year          1000 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 47.0+ KB


In [18]:
selected_df = df[["title", "Year", "citedby"]].rename(columns={"Year": "year"})

In [19]:
selected_df2 = df2[["title", "Year", "citedby"]].rename(columns={"Year": "year"})

In [74]:
combined_df = pd.concat([selected_df, selected_df2], ignore_index=True)
# แปลงตัวอักษรในคอลัมน์ 'title' ให้เป็นตัวเล็กทั้งหมด
combined_df['title'] = combined_df['title'].str.lower()
combined_df = combined_df.drop_duplicates()



In [75]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19517 entries, 0 to 19572
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   title    19517 non-null  object 
 1   year     19517 non-null  int64  
 2   citedby  19517 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 609.9+ KB


In [76]:
import re

# แยกคำในคอลัมน์ 'title' และลบคำที่ไม่ใช่ตัวอักษรภาษาอังกฤษ
combined_df['title'] = combined_df['title'].apply(lambda x: [word for word in x.split() if re.match('^[a-zA-Z]+$', word)])

# แสดงผลลัพธ์
print(combined_df)


                                                   title  year  citedby
0      [public, health, and, international, epidemiol...  2018      1.0
1      [flexible, printed, active, antenna, for, digi...  2018      1.0
2      [parametric, study, of, hydrogen, production, ...  2018     21.0
3      [superhydrophobic, coating, from, fluoroalkyls...  2018     37.0
4      [electrochemical, dna, sensor, using, pyrrolid...  2018     68.0
...                                                  ...   ...      ...
19568  [use, of, letibotulinumtoxina, for, aesthetic,...  2023      2.0
19569  [green, composite, sponge, of, natural, rubber...  2023      7.0
19570  [evaluation, of, foot, and, mouth, disease, co...  2023      3.0
19571  [tracing, the, new, variant, in, the, communit...  2023     23.0
19572  [utilisation, of, sunflower, marble, waste, na...  2023      2.0

[19517 rows x 3 columns]


In [77]:
# สร้าง dictionary mapping ปีเป็นค่าตัวเลข
year_mapping = {2018: 1, 2019: 2, 2020: 3, 2021: 4, 2022: 5, 2023: 6, 2024: 7}

# แปลงปีให้เป็นค่าตัวเลข
combined_df['year'] = combined_df['year'].map(year_mapping)

# แสดงผลลัพธ์
print(combined_df)


                                                   title  year  citedby
0      [public, health, and, international, epidemiol...     1      1.0
1      [flexible, printed, active, antenna, for, digi...     1      1.0
2      [parametric, study, of, hydrogen, production, ...     1     21.0
3      [superhydrophobic, coating, from, fluoroalkyls...     1     37.0
4      [electrochemical, dna, sensor, using, pyrrolid...     1     68.0
...                                                  ...   ...      ...
19568  [use, of, letibotulinumtoxina, for, aesthetic,...     6      2.0
19569  [green, composite, sponge, of, natural, rubber...     6      7.0
19570  [evaluation, of, foot, and, mouth, disease, co...     6      3.0
19571  [tracing, the, new, variant, in, the, communit...     6     23.0
19572  [utilisation, of, sunflower, marble, waste, na...     6      2.0

[19517 rows x 3 columns]


In [78]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer

# ใช้ title ที่แยกคำแล้ว (คุณทำไว้แล้ว)
# แปลง title เป็นข้อความเดียวเพื่อให้ CountVectorizer สามารถทำงานได้
combined_df['title'] = combined_df['title'].apply(lambda x: ' '.join(x))  # ถ้า 'title' เป็น list ของคำ

# แปลง 'title' ด้วย CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(combined_df['title'])

# กำหนด 'citedby' เป็น target
y = combined_df['citedby']

# สร้างโมเดลและฝึกด้วยข้อมูล
model = LinearRegression()
model.fit(X, y)

# ทำนายค่า 'citedby' สำหรับแต่ละแถวใน DataFrame
predicted_citedby = model.predict(X)

# แปลงคำที่แยกจาก 'title' เป็นคอลัมน์
feature_names = vectorizer.get_feature_names_out()

# สร้าง DataFrame ที่เก็บคำและค่าทำนาย 'citedby' สำหรับแต่ละคำ
words_and_predicted_citedby = []

# ทำนายค่า citedby สำหรับแต่ละคำ
for i, word in enumerate(feature_names):
    # คำนวณค่า predicted_citedby สำหรับทุกแถวที่มีคำนี้
    word_indices = np.where(X[:, i].toarray() > 0)[0]  # หาแถวที่มีคำนี้
    word_predicted_citedby = predicted_citedby[word_indices]
    
    # เก็บคำและค่าทำนาย
    for j, value in zip(word_indices, word_predicted_citedby):
        words_and_predicted_citedby.append({
            'word': word,
            'predicted_citedby': value
        })

# สร้าง DataFrame จากข้อมูล
df_words_citedby = pd.DataFrame(words_and_predicted_citedby)

# คำนวณค่าเฉลี่ยของ predicted_citedby สำหรับแต่ละคำที่ซ้ำกัน
df_avg_citedby = df_words_citedby.groupby('word')['predicted_citedby'].mean().reset_index()

# แสดงผลตารางที่ไม่มีคำซ้ำ
print(df_avg_citedby)


               word  predicted_citedby
0             aaron          -0.006660
1               aas           2.992919
2                ab           6.769951
3           abalone           1.672210
4         abamectin           0.989324
...             ...                ...
22059            zu          -0.016016
22060   zuckerkandl           0.992778
22061      zwischen          27.986387
22062  zwitterionic           5.004296
22063       zymogen          -0.007215

[22064 rows x 2 columns]


In [81]:
df_avg_citedby.to_csv("samplepredict.csv")